![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/data-ingestion.png)

# Data Ingestion


In [ ]:
import azureml.dataprep as dprep

Data Prep has the ability to load different types of input data. You can use auto-reading functionality to detect the type of a file, or directly specify a file type and its parameters.

## Table of Contents
[Read Lines](#lines)<br>
[Read CSV](#csv)<br>
[Read Compressed CSV](#compressed-csv)<br>
[Read Excel](#excel)<br>
[Read Fixed Width Files](#fixed-width)<br>
[Read Parquet](#parquet)<br>
[Read Part Files Using Globbing](#globbing)<br>
[Read JSON](#json)<br>
[Read SQL](#sql)<br>
[Read PostgreSQL](#postgresql)<br>
[Read From Azure Blob](#azure-blob)<br>
[Read From ADLS](#adls)<br>
[Read From ADLSGen2](#adlsgen2)<br>
[Read Pandas DataFrame](#pandas-df)<br>
[Read From HTTP/HTTPS Link](#http)<br>

<a id="lines"></a>

## Read Lines

One of the simplest ways to read data using Data Prep is to just read it as text lines.

In [ ]:
dflow = dprep.read_lines(path='../data/crime.txt')
dflow.head(5)

With ingestion done, you can go ahead and start prepping the dataset.

In [ ]:
df = dflow.to_pandas_dataframe()
df

<a id="csv"></a>

## Read CSV

When reading delimited files, the only required parameter is `path`. Other parameters (e.g. separator, encoding, whether to use headers, etc.) are available to modify default behavior.
In this case, you can read a file by specifying only its location, then retrieve the first 5 rows to evaluate the result.

In [ ]:
dflow_duplicate_headers = dprep.read_csv(path='../data/crime_duplicate_headers.csv')
dflow_duplicate_headers.head(5)

From the result, you can see that the delimiter and encoding were correctly detected. Column headers were also detected. However, the first line seems to be a duplicate of the column headers. One of the parameters is a number of lines to skip from the files being read. You can use this to filter out the duplicate line.

In [ ]:
dflow_skip_headers = dprep.read_csv(path='../data/crime_duplicate_headers.csv', skip_rows=1)
dflow_skip_headers.head(5)

Now the data set contains the correct headers and the extraneous row has been skipped by `read_csv`. Next, look at the data types of the columns.

In [ ]:
dflow_skip_headers.dtypes

Unfortunately, all of the columns came back as strings. This is because, by default, Data Prep will not change the type of the data. Since the data source is a text file, all values are kept as strings. In this case, however, numeric columns should be parsed as numbers. To do this, set the `infer_column_types` parameter to `True`, which will trigger type inference to be performed.


In [ ]:
dflow_inferred_types = dprep.read_csv(path='../data/crime_duplicate_headers.csv',
                          skip_rows=1,
                          infer_column_types=True)
dflow_inferred_types.dtypes

Now several of the columns were correctly detected as numbers and their `FieldType` is Decimal.

With ingestion done, the data set is ready to start preparing.

In [ ]:
df = dflow_inferred_types.to_pandas_dataframe()
df

<a id="compressed-csv"></a>

## Read Compressed CSV

Data Prep can also read delimited files compressed in an archive. The `archive_options` parameter specifies the type of archive and glob pattern of entries in the archive.

At this moment, only reading from ZIP archives is supported.

In [ ]:
from azureml.dataprep import ArchiveOptions, ArchiveType

dflow = dprep.read_csv(path='../data/crime.zip',
                          archive_options=ArchiveOptions(archive_type=ArchiveType.ZIP, entry_glob='*10-20.csv'))
dflow.head(5)

<a id="excel"></a>

## Read Excel

Data Prep can also load Excel files using the `read_excel` method.

In [ ]:
dflow_default_sheet = dprep.read_excel(path='../data/crime.xlsx')
dflow_default_sheet.head(5)

Here, the first sheet of the Excel document has been loaded. You could achieve the same result by specifying the name of the desired sheet explicitly.

In [ ]:
dflow_second_sheet = dprep.read_excel(path='../data/crime.xlsx', sheet_name='Sheet2')
dflow_second_sheet.head(5)

As you can see, the table in the second sheet had headers as well as three empty rows, so you can modify the arguments accordingly.

In [ ]:
dflow_skipped_rows = dprep.read_excel(path='../data/crime.xlsx',
                                      sheet_name='Sheet2',
                                      use_column_headers=True,
                                      skip_rows=3)
dflow_skipped_rows.head(5)

In [ ]:
df = dflow_skipped_rows.to_pandas_dataframe()
df

You can see in the results that the FBI Code column now contains some NaN values where before, when calling head, it didn't. By default, `to_pandas_dataframe` attempts to coalesce columns into a single type for better performance and lower memory overhead. This specific column has a mixutre of both numbers and strings and the strings were replaced with NaN values.

If you wish to keep the mixed-type column in the Pandas DataFrame, you can set the `extended_types` argument to True when calling `to_pandas_dataframe`.

In [ ]:
df = dflow_skipped_rows.to_pandas_dataframe(extended_types=True)
df

<a id="fixed-width"></a>

## Read Fixed Width Files

For fixed-width files, you can specify a list of offsets. The first column is always assumed to start at offset 0.

In [ ]:
dflow_fixed_width = dprep.read_fwf('../data/crime.txt', offsets=[8, 17, 26, 33, 56, 58, 74])
dflow_fixed_width.head(5)

Looking at the data, you can see that the first row was used as headers. In this particular case, however, there are no headers in the file, so the first row should be treated as data.

Passing in `PromoteHeadersMode.NONE` to the `header` keyword argument avoids header detection and gets the correct data.

In [ ]:
dflow_no_headers = dprep.read_fwf('../data/crime.txt',
                          offsets=[8, 17, 26, 33, 56, 58, 74],
                          header=dprep.PromoteHeadersMode.NONE)
dflow_no_headers.head(5)

In [ ]:
df = dflow_no_headers.to_pandas_dataframe()
df

<a id="parquet"></a>

## Read Parquet

Data Prep has two different methods for reading data stored as Parquet.

Currently, both methods require the `pyarrow` package to be installed in your Python environment. This can be done via `pip install azureml-dataprep[parquet]`.

### Read Parquet File

For reading single `.parquet` files, or a folder full of only Parquet files, use `read_parquet_file`.

In [ ]:
dflow = dprep.read_parquet_file('../data/crime.parquet')
dflow.head(5)

Parquet data is explicitly typed so no type inference is needed.

In [ ]:
dflow.dtypes

### Read Parquet Dataset

A Parquet Dataset is different from a Parquet file in that it could be a folder containing a number of Parquet files within a complex directory structure. It may have a hierarchical structure that partitions the data by value of a column. These more complex forms of Parquet data are commonly produced by Spark/HIVE.

For these more complex data sets, you can use `read_parquet_dataset`, which uses pyarrow to handle complex Parquet layouts. This will also handle single Parquet files, though these are better read using `read_parquet_file`.

In [ ]:
dflow = dprep.read_parquet_dataset('../data/parquet_dataset')
dflow.head(5)

The above data was partitioned by the value of the `Arrest` column. It is a boolean column in the original crime0 data set and hence was partitioned by `Arrest=true` and `Arrest=false`.

The directory structure is printed below for clarity.

In [ ]:
import os
for path, dirs, files in os.walk('../data/parquet_dataset'):
    level = path.replace('../data/parquet_dataset', '').count(os.sep)
    indent = '   ' * (level)
    print(indent + os.path.basename(path) + '/')
    fileindent = '   ' * (level + 1)
    for f in files:
        print(fileindent + f)

<a id="globbing"></a>

## Read Part Files Using Globbing

Data Prep supports globbing, which allows you to read partitioned files (or any other type of files) in a folder. Globbing is supported by all of the read transformations that take in file paths, such as `read_csv`, `read_lines`, etc. By specifying `../data/crime_partfiles/part-*` in the path, we will read all files start with `part-`in `crime_partfiles` folder and return them in one Dataflow. [`auto_read_file`](./auto-read-file.ipynb) will detect column types of your part files and parse them automatically.

In [ ]:
dflow_partfiles = dprep.auto_read_file(path='../data/crime_partfiles/part-*')
dflow_partfiles.head(5)

<a id="json"></a>

## Read JSON

Data Prep can also load JSON files.

In [ ]:
dflow_json = dprep.read_json(path='../data/json.json')
dflow_json.head(15)

When you use `read_json`, Data Prep will attempt to extract data from the file into a table. You can also control the file encoding Data Prep should use as well as whether Data Prep should flatten nested JSON arrays.

Choosing the option to flatten nested arrays could result in a much larger number of rows.

In [ ]:
dflow_flat_arrays = dprep.read_json(path='../data/json.json', flatten_nested_arrays=True)
dflow_flat_arrays.head(5)

<a id="sql"></a>

## Read SQL

Data Prep can also fetch data from SQL servers. Currently, only Microsoft SQL Server is supported.

To read data from a SQL server, first create a data source object that contains the connection information.

In [ ]:
secret = dprep.register_secret(value="dpr3pTestU$er", id="dprepTestUser")
ds = dprep.MSSQLDataSource(server_name="dprep-sql-test.database.windows.net",
                           database_name="dprep-sql-test",
                           user_name="dprepTestUser",
                           password=secret)

As you can see, the password parameter of `MSSQLDataSource` accepts a Secret object. You can get a Secret object in two ways:
1. Register the secret and its value with the execution engine.
2. Create the secret with just an id (useful if the secret value was already registered in the execution environment).

Now that you have created a data source object, you can proceed to read data.

In [ ]:
dflow = dprep.read_sql(ds, "SELECT top 100 * FROM [SalesLT].[Product]")
dflow.head(5)

In [ ]:
df = dflow.to_pandas_dataframe(extended_types=True)
df.dtypes

<a id="postgresql"></a>

## Read PostgreSQL

Data Prep can also fetch data from Azure PostgreSQL servers.

To read data from a PostgreSQL server, first create a data source object that contains the connection information.

In [ ]:
secret = dprep.register_secret(value="dpr3pTestU$er", id="dprepPostgresqlUser")
ds = dprep.PostgreSQLDataSource(server_name="dprep-postgresql-test.postgres.database.azure.com",
                                database_name="dprep-postgresql-testdb",
                                user_name="dprepPostgresqlReadOnlyUser@dprep-postgresql-test",
                                password=secret)

As you can see, the password parameter of `PostgreSQLDataSource` accepts a Secret object as well.
Now that you have created a PostgreSQL data source object, you can proceed to read data.

In [ ]:
dflow = dprep.read_postgresql(ds, "SELECT * FROM public.people")
dflow.head(5)

In [ ]:
dflow.dtypes

<a id="azure-blob"></a>

## Read from Azure Blob

You can read files stored in public Azure Blob by directly passing a file url. To read file from a protected Blob, pass SAS (Shared Access Signature) URI with both resource URI and SAS token in the path.

In [ ]:
dflow = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/testfiles/read_csv_duplicate_headers.csv', skip_rows=1)
dflow.head(5)

<a id="adls"></a>

## Read from ADLS

Data Prep currently supports both ADLS and ADLSGen2. There are two ways the Data Prep API can acquire the necessary OAuth token to access Azure DataLake Storage:
1. Retrieve the access token from a recent login session of the user's [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) login.
2. Use a ServicePrincipal (SP) and a certificate as a secret.

### Using Access Token from a recent Azure CLI session

On your local machine, run the following command:
```
az login
```
If your user account is a member of more than one Azure tenant, you need to specify the tenant, either in the AAD url hostname form '<your_domain>.onmicrosoft.com' or the tenantId GUID. The latter can be retrieved as follows:
```
az account show --query tenantId
```

```python
dflow = read_csv(path = DataLakeDataSource(path='adl://dpreptestfiles.azuredatalakestore.net/crime-spring.csv', tenant='microsoft.onmicrosoft.com'))
head = dflow.head(5)
head
```

### Create a ServicePrincipal via Azure CLI

A ServicePrincipal and the corresponding certificate can be created via [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest).
This particular SP is configured as Reader, with its scope reduced to just the ADLS account 'dpreptestfiles'.
```
az account set --subscription "Data Wrangling development"
az ad sp create-for-rbac -n "SP-ADLS-dpreptestfiles" --create-cert --role reader --scopes /subscriptions/35f16a99-532a-4a47-9e93-00305f6c40f2/resourceGroups/dpreptestfiles/providers/Microsoft.DataLakeStore/accounts/dpreptestfiles
```
This command emits the appId and the path to the certificate file (usually in the home folder). The .crt file contains both the public certificate and the private key in PEM format.

Extract the thumbprint with:
```
openssl x509 -in adls-dpreptestfiles.crt -noout -fingerprint
```

### Configure ADLS Account for ServicePrincipal

To configure the ACL for the ADLS filesystem, use the objectId of the user or, here, ServicePrincipal:
```
az ad sp show --id "fbc406bf-f7c2-410d-bc26-8b08e4dab1aa" --query objectId
```
Configure Read and Execute access for the ADLS file system. Since the underlying HDFS ACL model doesn't support inheritance, folders and files need to be ACL-ed individually.
```
az dls fs access set-entry --account dpreptestfiles --acl-spec "user:999a21ef-75aa-4538-b325-249285672204:r-x" --path /
az dls fs access set-entry --account dpreptestfiles --acl-spec "user:999a21ef-75aa-4538-b325-249285672204:r--" --path /crime-spring.csv
```

References:
- [az ad sp](https://docs.microsoft.com/en-us/cli/azure/ad/sp?view=azure-cli-latest)
- [az dls fs access](https://docs.microsoft.com/en-us/cli/azure/dls/fs/access?view=azure-cli-latest)
- [ACL model for ADLS](https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/data-lake-store/data-lake-store-access-control.md)

In [ ]:
certThumbprint = '84:60:1E:BE:AA:FF:90:A0:7A:73:38:80:F7:D0:12:44:98:70:9C:3A'
certificate = ''
with open('../data/adls-dpreptestfiles.crt', 'rt', encoding='utf-8') as crtFile:
    certificate = crtFile.read()

servicePrincipalAppId = "fbc406bf-f7c2-410d-bc26-8b08e4dab1aa"

### Acquire an OAuth Access Token

Use the adal package (via: `pip install adal`) to create an authentication context on the MSFT tenant and acquire an OAuth access token. Note that for ADLS, the `resource` in the token request must be for 'datalake.azure.net', which is different from most other Azure resources.

In [ ]:
import adal
from azureml.dataprep.api.datasources import DataLakeDataSource

ctx = adal.AuthenticationContext('https://login.microsoftonline.com/microsoft.onmicrosoft.com')
token = ctx.acquire_token_with_client_certificate('https://datalake.azure.net/', servicePrincipalAppId, certificate, certThumbprint)
dflow = dprep.read_csv(path = DataLakeDataSource(path='adl://dpreptestfiles.azuredatalakestore.net/crime-spring.csv', accessToken=token['accessToken']))
dflow.to_pandas_dataframe().head()

<a id="adlsgen2"></a>

## Read from ADLSGen2

Please refer to the Read for ADLS section above to get details of how to register a Service Principal and obtain an OAuth access token.[ADLS](http://localhost:8888/notebooks/notebooks/how-to-guides/data-ingestion.ipynb#adls)

### Configure ADLSGen2 Account for ServicePrincipal

In [ ]:
certThumbprint = '23:66:84:6B:3A:14:9E:B1:17:CA:EE:E3:BB:2C:21:2D:20:B0:DF:F2'
certificate = ''
with open('../data/ADLSgen2-datapreptest.crt', 'rt', encoding='utf-8') as crtFile:
    certificate = crtFile.read()

servicePrincipalAppId = "127a58c3-f307-46a1-969e-a6b63da3f411"

### Acquire an OAuth Access Token for ADLSGen2

In [ ]:
import adal
from azureml.dataprep.api.datasources import ADLSGen2

ctx = adal.AuthenticationContext('https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47')
token = ctx.acquire_token_with_client_certificate('https://storage.azure.com/', servicePrincipalAppId, certificate, certThumbprint)
dflow = dprep.read_csv(path = ADLSGen2(path='https://adlsgen2datapreptest.dfs.core.windows.net/datapreptest/people.csv', accessToken=token['accessToken']))
dflow.to_pandas_dataframe().head()

<a id="pandas-df"></a>

## Read Pandas DataFrame

There are situations where you may already have some data in the form of a pandas DataFrame.
The steps taken to get to this DataFrame may be non-trivial or not easy to convert to Data Prep Steps. The `read_pandas_dataframe` reader can take a DataFrame and use it as the data source for a Dataflow.

You can pass in a path to a directory (that doesn't exist yet) for Data Prep to store the contents of the DataFrame; otherwise, a temporary directory will be made in the system's temp folder. The files written to this directory will be named `part-00000` and so on; they are written out in Data Prep's internal row-based file format.

In [ ]:
dflow = dprep.read_excel(path='../data/crime.xlsx')
dflow = dflow.drop_columns(columns=['Column1'])
df = dflow.to_pandas_dataframe()
df.head(5)

After loading in the data you can now do `read_pandas_dataframe`. If you only need to consume the Dataflow created from the current environment, you can read the DataFrame in memory.

In [ ]:
dflow_df = dprep.read_pandas_dataframe(df, in_memory=True)
dflow_df.head(5)

However, if you intend to use this Dataflow past the end of your current Python session (such as by saving the Dataflow to a file), you can provide a cache directory where the contents of the DataFrame will be stored so they can be retrieved later.

In [ ]:
import shutil
cache_dir = 'dflow_df'
shutil.rmtree(cache_dir, ignore_errors=True)
dflow_df = dprep.read_pandas_dataframe(df, cache_dir)

In [ ]:
dflow_df.head(5)

<a id="http"></a>

# Read from HTTP/HTTPS Link

You can pass in an HTTP/HTTPS path when loading remote data source.

In [ ]:
dflow = dprep.read_csv('https://dprepdata.blob.core.windows.net/test/Sample-Spreadsheet-10-rows.csv')
dflow.head(5)